# Interactive Hurricane/Topical Storm Tracker using Folium

by [Ken Urquhart](https://linkedin.com/in/kenu)

Publicly available data from the National Hurricane Center:

https://www.nhc.noaa.gov/gis/

provides a lot of useful information about tropical storms and hurricanes that you can create yourself instead of waiting for updates from news services.

Find out if you are in the latest predicted path? What the winds might be? Are you in a storm surge area?

This notebook shows you how to:

* Work with files no matter what OS you are running on using `pathlib`

* Work with files (and ZIP'd files) located on Internet servers using `urllib` and `zipfile`

* Work with `shapely` and `KML` geographic data using `geopandas`...
* ...including the (lesser known) tricks for working with `KML` and `KMZ` geographic files

* Create interactive maps or geographic data using `folium` and `folium plugins`
* Work with color maps and tooltips to annotate maps with geographic information
* Build layered maps when you have a lot of information to visualize

* Use of `regular expressions` to clean up data columns in `pandas` and `geopandas`

So let's get started...

Load all the libraries we are going to use.

In [ ]:
import matplotlib.pyplot as plt
import datetime
import folium
from folium import plugins
import geopandas as gpd
import fiona
from shapely.geometry import Polygon, box
import numpy as np
import pandas as pd

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import shutil
from pathlib import Path

pd.set_option('display.max_columns', None)

# >> Artemis 1 and Tropical Storm / Hurricane Ian

As of Sunday September 25, 2022, Tropical Storm Ian is threatening to become a hurricane and may pass near the Kennedy Space Center (KSC). Artemis 1 is on Launch Pad 39B and may be returned to the Vehicle Assembly Building to ride out the storm.

I've added a marker for Launch Pad 39B so you can see how close TS/Hurricane Ian may come to the KSC and what kinds of winds are expected.

## Storm reference code

The National Hurricane Center identifies all digital storm information by two-letter and 6 digit code. The first two letters indicate which ocean the storm is in (e.g. **al** means Atlantic and **ep** means Eastern Pacific). This is followed by a 2-digit storm number and the 4-digit year. Storm numbers begin with **01**.

So data files associated with Tropical Storms and Hurricanes have codes like:

In [ ]:
storm = 'al092022'
storm_name = 'TS Ian'

## About map projections...

The world is a sphere (more or less) and we visualize it using flat maps. That means having to project a part of a sphere onto a flat surface. We use different projections to minimize distortion of the area on the world sphere we are making the map of. A full discussion of projections is a notebook in itself. You can read more about it here:

https://source.opennews.org/articles/choosing-right-map-projection/

The best projection to render North America reasonably well on a flat surface is 9311. All `folium` maps assume a projection of `EPSG:4326`. While we work with U.S. map data in the `EPSG;9311` projection, we need to re-project them into the folium default for display.

In [ ]:
epsg_project = 'EPSG:9311'
epsg_folium = 'EPSG:4326'

# Get the storm data from the NHS

Publicly available storm data is available for download at:

https://www.nhc.noaa.gov/gis/

There are 5 interesting data products:

1. Advisory forecast track: the "most likely" path the storm will take (as a line and as a set of points ordered by by date and time)
2. Storm cone of uncertainty: region the path of the storm could take over the next few days
3. Storm surge: lines along the coast where the storm can cause sea water to surge and cause flooding
4. Arrival Time of Tropical Storm Force Winds: best estimate when storm winds will arrive at a given location
5. Wind field probabilities: estimate of how high wind speeds could be over the next 5 days as the storm moves

Let's get the storm data...

In [ ]:
sdir = Path('raw_data')
if sdir.is_dir():
    shutil.rmtree(sdir)
sdir.mkdir()

In [ ]:
zipurl = f'https://www.nhc.noaa.gov/gis/forecast/archive/{storm}_5day_latest.zip'
zipurl

In [ ]:
subFolder = sdir / f'{storm}_5day_latest/'
print(subFolder)
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall(subFolder)

## 1. Advisory forecast track (as a line and date/time points)

In [ ]:
for file in subFolder.glob('*_5day_lin.shp'):
    track_line = file
    break
track_line

Peel off the storm prediction run number and use it for output folder and file naming...

In [ ]:
t = str(track_line)
t_s = storm + '-'
t_e = '_5day_lin.shp'
gen = t[t.find(t_s)+len(t_s):t.rfind(t_e)]
gen

Date and time stamp for this run

In [ ]:
dateStr = datetime.datetime.today().strftime('%Y%m%d') + f'_{gen}_{storm_name}'
dateStr

In [ ]:
pathStr = Path(dateStr)
pathStr

Create the output directory for this storm prediction run.

In [ ]:
if pathStr.is_dir():
    print ("Directory '%s' already exists" % pathStr)
else:
    try:
        pathStr.mkdir()
    except OSError:
        print ("Failed to create directory '%s'" % pathStr)
    else:
        print ("Created directory '%s'" % pathStr)

### Advisory storm track line data

You can display the data in any `geopandas` dataframe by calling `.plot()`

In [ ]:
gdf_storm_line = gpd.read_file(track_line)
gdf_storm_line.crs = epsg_folium
gdf_storm_line.plot()

### Advisory storm track points data

In [ ]:
for file in subFolder.glob('*_5day_pts.shp'):
    track_pts = file
    break
track_pts

gdf_storm_points = gpd.read_file(track_pts)
gdf_storm_points.crs = epsg_folium
gdf_storm_points.plot()

In [ ]:
gdf_storm_points.head(2)

## 2. Storm cone of uncertainty

Software that calculates the path of tropical storms and hurricanes uses statistics to determine probable paths of the eye of the storm. The eye can wander around anywhere inside that cone of uncertainty. The farther in the future the projected storm path, the wider the uncertaintly. So you get this ice cream cone shaped (comma shaped?) envelope surrounding the *most probable* eye path downloaded in section 1.

In [ ]:
for file in subFolder.glob('*_5day_pgn.shp'):
    track_env = file
    break
track_env

In [ ]:
gdf_storm_envelope_1 = gpd.read_file(track_env)
gdf_storm_envelope_1.crs = epsg_folium
gdf_storm_envelope_1.plot()

Since we want the map we make to center around this storm envelope, we need to calculate the centroid and then center the map around it.

In [ ]:
def getXY(pt):
    return (pt.x, pt.y)

gdf_storm_envelope_1['geometry'].crs = epsg_project
centroidseries = gdf_storm_envelope_1['geometry'].centroid
gdf_storm_envelope_1['geometry'].crs = epsg_folium

longs,lats = [list(t) for t in zip(*map(getXY, centroidseries))]
print(lats)
print(longs)

## 3. Storm surge

Areas of probabaly flooding due to the storm are also included in the storm data.

In [ ]:
do_surge = False

for file in subFolder.glob('*_ww_wwlin.shp'):
    track_surge = file
    display(track_surge)
    do_surge = True
    break
else:
    print('No surge data')
print('do_surge = ', do_surge)

In [ ]:
try:
    gdf_wwlin = gpd.read_file(track_surge)
    gdf_wwlin.crs = epsg_folium
    gdf_wwlin.plot()
except:
    do_surge = False
    
print(do_surge)

## 4. Arrival Time of Tropical Storm Force Winds

Reading a KMZ file into geopandas...is not a straightforward process (though that may change in a future update).

* Enable `fiona` to read `KML` files
* Download the KMZ file and unzip it into a directory
* Read the KML file in that directory to a `geopandas` file

In [ ]:
do_wind = True

In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
zipurl = f'https://www.nhc.noaa.gov/storm_graphics/api/{storm.upper()}_most_likely_toa_34_latest.kmz'
print(zipurl)

In [ ]:
outKML = Path(sdir / str(storm.upper() + '_kml/'))
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall(outKML)

In [ ]:
for file in outKML.glob('*.kml'):
    inKML = file
    break
inKML

In [ ]:
gdf_winds = gpd.read_file(inKML, driver='KML')
gdf_winds = gdf_winds[gdf_winds['geometry'].geom_type == 'LineString']
gdf_winds

In [ ]:
import re
gdf_winds['Description'] = gdf_winds['Description'].apply(lambda x: re.sub('<[^<]+?>', '', x).strip())

In [ ]:
gdf_winds.reset_index(drop=True, inplace=True)
gdf_winds

In [ ]:
gdf_winds.plot()

In [ ]:
pathStr / (str(pathStr) + "_Projected_Path.html")

## 5. Wind field probabilities

In [ ]:
zipurl = 'https://www.nhc.noaa.gov/gis/forecast/archive/wsp_120hr5km_latest.zip'
print(zipurl)

In [ ]:
windDir =  sdir / 'wsp_120hr5km_latest'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall(windDir)

In [ ]:
wind_file = []
for file in windDir.glob('*knt*.shp'):
   wind_file.append(file) 
wind_file.sort()
wind_file

In [ ]:
gdf_wind_field = []
for f in wind_file:
    gwf = gpd.read_file(f)
    gwf.crs = epsg_folium
    gdf_wind_field.append(gwf)
    gwf.plot()

In [ ]:
gdf_wind_field[0]

Create a color map for the wind fields...

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx

cm = plt.get_cmap('RdYlGn_r')#'Spectral_r') 
cNorm  = colors.Normalize(vmin=0, vmax=11)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)

# Folium Maps

Generate a new folium map and add a marker for Launch Pad 39B where Artemis 1 launches from.

Values of `tiles` variable in `Map` call:

* “OpenStreetMap”
* “Mapbox Bright” (Limited levels of zoom for free tiles)
* “Mapbox Control Room” (Limited levels of zoom for free tiles)
* “Stamen” (Terrain, Toner, and Watercolor)
* “Cloudmade” (Must pass API key)
* “Mapbox” (Must pass API key)
* “CartoDB” (positron and dark_matter)

In [ ]:
#do_surge = False
#do_wind = True

folium_map = folium.Map(location=[lats[0],longs[0]], zoom_start=4)#, tiles="CartoDB positron")

#-------------------------

g_storm_track = folium.map.FeatureGroup(name="Storm Track", overlay=True, control=True, show=True)
g_eye_zone = folium.map.FeatureGroup(name="Eye Zone", overlay=True, control=True, show=True)
g_surge = folium.map.FeatureGroup(name="Storm Surge", overlay=True, control=True, show=True)
g_wind = folium.map.FeatureGroup(name="Wind Arrival Times", overlay=True, control=True, show=False)

g_wind_prob_34 = folium.map.FeatureGroup(name="Wind Prob 34kt", overlay=True, control=True, show=False)
g_wind_prob_50 = folium.map.FeatureGroup(name="Wind Prob 50kt", overlay=True, control=True, show=False)
g_wind_prob_64 = folium.map.FeatureGroup(name="Wind Prob 64kt", overlay=True, control=True, show=False)

#-------------------------

g_storm_track.add_child(
    folium.features.GeoJson(
        gdf_storm_line,
        style_function=lambda feature: {
            'color' : '#0000ff',
            'weight' : 4,
            'fillOpacity' : 0.2
        }
    ))

for idx, row in gdf_storm_points.iterrows():
    tooltip_text = 'Date: ' + row.FLDATELBL + '<br>Strength: ' + row.STORMTYPE
    marker = folium.CircleMarker(
        [row.LAT, row.LON],
        radius=8,
        color='blue',
        fill=True,
        fill_opacity=0.3,
        weight=2,
        tooltip = tooltip_text)
    g_storm_track.add_child(marker)

#-------------------------

g_eye_zone.add_child(
    folium.features.GeoJson(
        gdf_storm_envelope_1,
        style_function=lambda feature: {
            'fillColor': 'blue',
            'color' : '#0000ff',
            'weight' : 3,
            'fillOpacity' : 0.2
        }
    ))

#-------------------------

if do_surge:
    g_surge.add_child(
        folium.features.GeoJson(
            gdf_wwlin,
            tooltip = 'Storm Surge',
            style_function=lambda feature: {
                'color' : '#0000ff',
                'weight' : 4,
                'fillOpacity' : 0.3
            }
        )
    )

#-------------------------

if do_wind:
    g_wind.add_child(
        folium.features.GeoJson(
            gdf_winds,
            tooltip = folium.features.GeoJsonTooltip(fields=['Description'], aliases=['Wind arrives:']),
            style_function=lambda feature: {
                'color' : '#0000ff',
                'weight' : 2,
                'fillOpacity' : 0.3
            }
        )
    )

#-------------------------

def styler(feature):
    p = int(feature['id'])#feature['properties']['PERCENTAGE']
    if p < 0:
        p = -1
    if p < 0:
        fill_opacity = 0
        line_weight = 0
    else:
        fill_opacity = 0.3
        line_weight = 2
    color_string = colors.to_hex(scalarMap.to_rgba(float(p)))
    return {
        'fillColor': color_string,
        'color' : color_string,
        'weight' : line_weight,
        'fillOpacity' : fill_opacity
    }

g_wind_prob_34.add_child(
    folium.features.GeoJson(
        gdf_wind_field[0][gdf_wind_field[0]['geometry'] != None],
        tooltip = folium.features.GeoJsonTooltip(fields=['PERCENTAGE'], aliases=['Prob 34 knot winds:']),
        style_function=styler
    )
)

g_wind_prob_50.add_child(
    folium.features.GeoJson(
        gdf_wind_field[1][gdf_wind_field[1]['geometry'] != None],
        tooltip = folium.features.GeoJsonTooltip(fields=['PERCENTAGE'], aliases=['Prob 50 knot winds:']),
        style_function=styler
    )
)

g_wind_prob_64.add_child(
    folium.features.GeoJson(
        gdf_wind_field[2][gdf_wind_field[2]['geometry'] != None],
        tooltip = folium.features.GeoJsonTooltip(fields=['PERCENTAGE'], aliases=['Prob 64 knot winds:']),
        style_function=styler
    )
)

#-------------------------

folium_map.add_child(g_eye_zone)
if do_surge:
    folium_map.add_child(g_surge)
folium_map.add_child(g_wind_prob_34)
folium_map.add_child(g_wind_prob_50)
folium_map.add_child(g_wind_prob_64)
if do_wind:
    folium_map.add_child(g_wind)
folium_map.add_child(g_storm_track)

#-------------------------

folium_map.add_child(
    folium.Marker(
        location=[28.6272, -80.6208],
        popup='Artemis 1 Launchpad 39B',
        tooltip='Artemis 1 Launchpad 39B'
    )
)

#-------------------------

folium.LayerControl(collapsed=False).add_to(folium_map)

#-------------------------

folium_map

In [ ]:
outMap = pathStr / (str(pathStr) + "_Projected_Path.html")
folium_map.save(str(outMap))
outMap